In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.insert(1, "../python/functions")
sys.path.insert(2, "../python/architecture")

In [3]:
from data_prep_functions import mnist_prep
from model_functions import *
from plotting_functions import *
import no_gpu
import reproducible

In [4]:
X_train, X_val, y_train, y_val = mnist_prep()

In [5]:
# use samller dataset for increased speed
X_train_small = X_train[:1000, :]
X_val_small = X_val[:500, :]
y_train_small = y_train[:1000]
y_val_small = y_val[:500]

n_cols = X_train_small.shape[1]

# EnKF

In [6]:
X_train = X_train_small
X_test = X_val_small
y_train = y_train_small
y_test = y_val_small

In [7]:
batch_size = 50
epochs = 5
particles = 10
early_stopping = 0.001
batch_normal = False
shuffle = True # noch einbauen!!!

In [8]:
layers = 5
neurons = [128, 128, 64, 32, 10]
n_cols = X_train.shape[1]

In [9]:
delta = 0.005
h_0 = 40
epsilon = 0.005

In [10]:
n = len(X_train)
num_batches = int(np.ceil(n / batch_size))
batch_indices = np.cumsum([0] + list(np.ones(num_batches) * batch_size))
batch_indices[-1] = n

In [11]:
X_batches = [X_train[int(batch_indices[i]):int(batch_indices[i+1])] for i in range(len(batch_indices)-1)]
y_batches = [y_train[int(batch_indices[i]):int(batch_indices[i+1])] for i in range(len(batch_indices)-1)]

In [12]:
model_dict = {}
weights_dict = {}
y_pred_dict = {}
jacobian_dict = {}
weights_vector_dict = {}

# init_model already has weights and biases following the Glorot distribution
# it can already be used to predict and evaluate, but it is very bad (<10% accuracy)
# only used to determine shapes and shape_elements via its weights
init_model = nn_model_structure(layers = layers,
                                neurons = neurons,
                                n_cols = n_cols)
init_model = nn_model_compile(init_model,
                              optimizer = "sgd")
weights = init_model.get_weights()
# shape contains the shapes of the weight matrices and bias vectors as a list of arrays
shapes = [np.array(params.shape) for params in weights]
# shape_elements contains the indices of the weights as a vector and tells where to cut
shape_elements = np.cumsum([0] + [np.prod(shape) for shape in shapes])

In [13]:
for i in range(particles):
    # just an initial model with the correct structure regarding neurons, layers, activation functions, Glorot initialization
    model = nn_model_structure(layers = layers,
                               neurons = neurons,
                               n_cols = n_cols)
    model = nn_model_compile(model,
                             optimizer = "sgd")
    # for every particle write the model in a dictionary
    model_dict["model_{}".format(str(i+1))] = model
    
    # for every particles write the weights and biases in a dictionary
    weights_dict["model_{}".format(str(i+1))] = model_dict["model_{}".format(str(i+1))]\
                                                    .get_weights()

In [14]:
for i in range(particles):
    print(model_dict["model_{}".format(str(i+1))].evaluate(X_val_small, y_val_small)[1])

16/16 [==============================] - 0s 563us/step - loss: 2.3359 - accuracy: 0.0900
0.09000000357627869
16/16 [==============================] - 0s 563us/step - loss: 2.3586 - accuracy: 0.0960
0.09600000083446503
16/16 [==============================] - 0s 563us/step - loss: 2.3362 - accuracy: 0.0900
0.09000000357627869
16/16 [==============================] - 0s 563us/step - loss: 2.3503 - accuracy: 0.1080
0.1080000028014183
16/16 [==============================] - 0s 563us/step - loss: 2.4043 - accuracy: 0.0880
0.08799999952316284
16/16 [==============================] - 0s 563us/step - loss: 2.3688 - accuracy: 0.0780
0.07800000160932541
16/16 [==============================] - 0s 563us/step - loss: 2.3208 - accuracy: 0.1360
0.13600000739097595
16/16 [==============================] - 0s 563us/step - loss: 2.4364 - accuracy: 0.0980
0.09799999743700027
16/16 [==============================] - 0s 563us/step - loss: 2.3174 - accuracy: 0.1340
0.1340000033378601
16/16 [==============

In [15]:
len(X_batches)

20

bei b=6 geht was schief -> vorher steigen die Accuracies (zumindest für manche Partikel), dann wieder die 8,4 %

da wird der Loss zu groß und dadurch kommen nan

da werden die Gewichte und Bias zu groß

kleineres $h_t$ hilft (Gewichte und Bias werden langsamer groß), aber Accuracy steigt weniger (vllt auch nur langsamer)

#### Lösungsidee: 1) Batch Normalization, 2) (muss sowieso auch noch gemacht werden) Mittelwerte der Partikel

### Nach dem ersten Schleifendurchlauf erhalten wir nicht mehr die Wahrscheinlichkeiten, sondern nur noch einen 0-1-Vektor. 
Zusätzliches .fit hilft nicht. Immer neue Modelle in erster particles-Schleife helfen nicht.

In [16]:
for b in range(6):    
    for i in range(particles):
        # for every particle write the predictions on the training batches in a dictionary
        y_pred_dict["model_{}".format(str(i+1))] = model_dict["model_{}".format(str(i+1))]\
                                                        .predict(X_batches[b])

        # for every particle write the Jacobian in a dictionary
        jacobian_dict["model_{}".format(str(i+1))] = (-1) * np.multiply(np.array(y_batches[b]), 
                                                                        np.array(1 / (y_pred_dict["model_{}".format(str(i+1))] + delta)))
    print(y_pred_dict["model_1"])
    # bis hier hin alles gut     
    # compute the mean of the predictions
    y_pred_mean = np.mean(list(y_pred_dict.values()), axis = 0)
    # bis hier ok
    # compute the matrix D elementwise
    d = np.zeros(shape = (particles, particles))
    for k in range(particles):
        y_pred_centered = y_pred_dict["model_{}".format(str(k+1))] - y_pred_mean
        for j in range(particles):
            d[k][j] = np.sum(np.multiply(y_pred_centered, jacobian_dict["model_{}".format(str(j+1))]))
                                    # d sieht recht einfach aus, aber das wird wohl stimmen
    # compute the scalar h_t
    h_t = h_0 / (np.sqrt(np.sum(d**2)) + epsilon)
    # bis hier wohl ok
    # Reshape the weights and biases so that they are no longer matrices and vectores, but now one single vector
    for i in range(particles):
        weights_array = np.array([])
        for j in range(len(weights_dict["model_{}".format(str(i+1))])):
            weights_array = np.append(weights_array, np.reshape(weights_dict["model_{}".format(str(i+1))][j], (1, -1)).ravel())
        weights_vector_dict["model_{}".format(str(i+1))] = weights_array
    # der Teil ist jetzt gedebugged    
    # matrix with particle parameters as row vectors
    weights_all_ptcls = np.array(list(weights_vector_dict.values()))

    # compute the matrix with the updates for each particle
    weights_all_ptcls = weights_all_ptcls - h_t * np.matmul(d, weights_all_ptcls)

    for i in range(particles):
        # write the updates back into the dictionary
        weights_vector_dict["model_{}".format(str(i+1))] = weights_all_ptcls[i]
        # reshape the updates, so that they are of the original matrx and vector shape
        for l in range(len(shape_elements)-1):
            start = shape_elements[l]
            end = shape_elements[l+1]
            weights_dict["model_{}".format(str(i+1))][l] = np.reshape(weights_vector_dict["model_{}".format(str(i+1))][start:end], tuple(shapes[l]))
        # set new weights for model
        model_dict["model_{}".format(str(i+1))].set_weights(weights_dict["model_{}".format(str(i+1))])

Instructions for updating:
Please use `model.predict()` instead.
[[0.08969521 0.09125192 0.14471631 0.10483738 0.10281324 0.10316657
  0.10425069 0.07176778 0.0806197  0.10688114]
 [0.08218075 0.08765528 0.13450104 0.09481126 0.10923961 0.11760543
  0.10356963 0.0544965  0.09163456 0.12430603]
 [0.09672759 0.08863358 0.14680548 0.08474655 0.10148374 0.10474286
  0.119863   0.05183246 0.09142287 0.1137419 ]
 [0.08609666 0.07721667 0.12806283 0.10249868 0.10793719 0.12640207
  0.10267585 0.06010416 0.08524907 0.12375674]
 [0.09441364 0.08408081 0.12376191 0.09253851 0.11749007 0.10970289
  0.09296862 0.06722209 0.09668078 0.12114068]
 [0.11641724 0.07979588 0.1366833  0.07699235 0.1265276  0.10056099
  0.09591322 0.0598855  0.09302006 0.11420383]
 [0.08666999 0.07766679 0.12446859 0.10231008 0.1156855  0.11812299
  0.10312534 0.05735904 0.09939921 0.1151925 ]
 [0.11322234 0.07653631 0.13658349 0.08500778 0.11121038 0.10405912
  0.11110285 0.0693581  0.074049   0.11887062]
 [0.0876643  0.

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1.

In [17]:
for i in range(particles):
    print(model_dict["model_{}".format(str(i+1))].evaluate(X_val_small, y_val_small)[1])

16/16 [==============================] - 0s 626us/step - loss: 140049878154722201731282705055744.0000 - accuracy: 0.1200
0.11999999731779099
16/16 [==============================] - 0s 563us/step - loss: 179745883034061673870524282654687232.0000 - accuracy: 0.0900
0.09000000357627869
16/16 [==============================] - 0s 563us/step - loss: 8175820246955383353036903546880000.0000 - accuracy: 0.1000
0.10000000149011612
16/16 [==============================] - 0s 626us/step - loss: 8485743487550635446378998892855296.0000 - accuracy: 0.1000
0.10000000149011612
16/16 [==============================] - 0s 563us/step - loss: 2311773640460428488363986581979136.0000 - accuracy: 0.1300
0.12999999523162842
16/16 [==============================] - 0s 563us/step - loss: 4387703919701491145761185035452416.0000 - accuracy: 0.1300
0.12999999523162842
16/16 [==============================] - 0s 563us/step - loss: 101793432585561392074827178621009920.0000 - accuracy: 0.0920
0.09200000017881393
16/

In [18]:
model_dict

{'model_1': <tensorflow.python.keras.engine.sequential.Sequential at 0x1a380072608>,
 'model_2': <tensorflow.python.keras.engine.sequential.Sequential at 0x1a380115508>,
 'model_3': <tensorflow.python.keras.engine.sequential.Sequential at 0x1a380146648>,
 'model_4': <tensorflow.python.keras.engine.sequential.Sequential at 0x1a3801768c8>,
 'model_5': <tensorflow.python.keras.engine.sequential.Sequential at 0x1a381175b08>,
 'model_6': <tensorflow.python.keras.engine.sequential.Sequential at 0x1a3811a6dc8>,
 'model_7': <tensorflow.python.keras.engine.sequential.Sequential at 0x1a3811dc0c8>,
 'model_8': <tensorflow.python.keras.engine.sequential.Sequential at 0x1a381208388>,
 'model_9': <tensorflow.python.keras.engine.sequential.Sequential at 0x1a38123b588>,
 'model_10': <tensorflow.python.keras.engine.sequential.Sequential at 0x1a38126d7c8>}

In [19]:
model_dict["model_1"].get_weights()

[array([[ 109351.54  ,   18089.766 , -486908.7   , ...,   25869.604 ,
          151569.33  , -398260.28  ],
        [ 270257.06  ,   67228.93  , -110691.4   , ..., -166657.47  ,
         -100774.586 , -493891.16  ],
        [  24114.545 ,  112240.53  ,  -40360.992 , ...,    3877.2693,
           72167.03  ,   61415.766 ],
        ...,
        [ 160075.53  , -139677.66  ,  266539.5   , ...,  -64734.17  ,
         -215182.47  ,  -34313.73  ],
        [   7043.6914,  -15887.845 ,  -42710.277 , ...,   26442.281 ,
         -106416.53  , -131934.45  ],
        [  37600.117 , -168354.61  , -123748.75  , ...,   31504.328 ,
          -90382.51  ,  -68723.74  ]], dtype=float32),
 array([ 503676.8  , -197197.03 ,   37364.664,  -69890.555, -543318.1  ,
         485067.25 ,    6026.515,  465090.38 ,  190716.33 ,   51536.516,
         268588.28 , -291804.44 , -216240.05 ,  -92722.016,  765121.06 ,
        -290379.03 ,  341754.8  ,   79792.984, -150192.84 ,  551449.7  ,
        -413822.2  , -520934.1

In [20]:
mean_weights = list(np.mean(list(weights_dict.values()), axis = 0))
init_model.set_weights(mean_weights)

In [21]:
init_model.evaluate(X_test, y_test)[0]

16/16 [==============================] - 0s 563us/step - loss: 2.3020 - accuracy: 0.0900


2.30204439163208

In [22]:
y_pred_dict["model_1"]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 

In [23]:
jacobian_dict["model_1"]

array([[  -0.        ,   -0.        ,   -0.        , -200.        ,
          -0.        ,   -0.        ,   -0.        ,   -0.        ,
          -0.        ,   -0.        ],
       [  -0.        , -200.        ,   -0.        ,   -0.        ,
          -0.        ,   -0.        ,   -0.        ,   -0.        ,
          -0.        ,   -0.        ],
       [  -0.        ,   -0.        , -200.        ,   -0.        ,
          -0.        ,   -0.        ,   -0.        ,   -0.        ,
          -0.        ,   -0.        ],
       [  -0.        ,   -0.        , -200.        ,   -0.        ,
          -0.        ,   -0.        ,   -0.        ,   -0.        ,
          -0.        ,   -0.        ],
       [  -0.        ,   -0.        ,   -0.        , -200.        ,
          -0.        ,   -0.        ,   -0.        ,   -0.        ,
          -0.        ,   -0.        ],
       [  -0.        ,   -0.        ,   -0.        , -200.        ,
          -0.        ,   -0.        ,   -0.        ,   -0